<div style="display: flex; justify-content: space-between;">
<a style="flex: 1; text-align: left;" href="./2_4_Significance.ipynb">← Previous: 2.4 Tests for the Significance of the Regression</a>
<a style="flex: 1; text-align: right;" href="./3_1_1_Bus.ipynb">Next: 3.1.1 Bus Data →</a>
</div>

### 2.5 Model Building
---


<div style="text-align: justify; line-height: 2">





<sup><a href="#footnote2">2</a></sup>

</div>
&nbsp;  
&nbsp;  

---

<a name="footnote1"></a>1: [PennState Eberly College of Science](./5_References.ipynb#1)  
<a name="footnote2"></a>2: [Ivan T. Ivanov](./5_References.ipynb#2)  
<a name="footnote3"></a>3: [OpenStax](./5_References.ipynb#3)  
<a name="footnote4"></a>4: [Heij, De Boer; Franses, Kloep; and Van Dijk](./5_References.ipynb#4)  
<a name="footnote5"></a>5: [Mostoufi, Navid; Constantinides, Alkis](./5_References.ipynb#5)

<div style="display: flex; justify-content: space-between;">
<a style="flex: 1; text-align: left;" href="./2_4_Significance.ipynb">← Previous: 2.4 Tests for the Significance of the Regression</a>
<span style="flex: 1; text-align: center;">2.5 Model Building</span>
<a style="flex: 1; text-align: right;" href="./3_1_1_Bus.ipynb">Next: 3.1.1 Bus Data→</a>
</div>
